In [1]:
import keras
from keras.optimizers import Adam

import yfinance as yf

ticker = yf.Ticker('AAPL')
ticker_history = ticker.history(period="max")
ticker_history

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12 00:00:00-05:00,0.099058,0.099488,0.099058,0.099058,469033600,0.0,0.0
1980-12-15 00:00:00-05:00,0.094321,0.094321,0.093890,0.093890,175884800,0.0,0.0
1980-12-16 00:00:00-05:00,0.087429,0.087429,0.086998,0.086998,105728000,0.0,0.0
1980-12-17 00:00:00-05:00,0.089152,0.089582,0.089152,0.089152,86441600,0.0,0.0
1980-12-18 00:00:00-05:00,0.091737,0.092167,0.091737,0.091737,73449600,0.0,0.0
...,...,...,...,...,...,...,...
2024-07-30 00:00:00-04:00,219.190002,220.330002,216.119995,218.800003,41643800,0.0,0.0
2024-07-31 00:00:00-04:00,221.440002,223.820007,220.630005,222.080002,50036300,0.0,0.0
2024-08-01 00:00:00-04:00,224.369995,224.479996,217.020004,218.360001,62501000,0.0,0.0


In [2]:
import numpy as np
import pandas as pd
import keras
from sklearn.preprocessing import StandardScaler

# Define the create_sequences function
def create_sequences(data, seq_length):
    data_array = []
    for i in range(len(data) - seq_length + 1):
        seq = data[i:i+seq_length]
        data_array.append(seq)
    return np.array(data_array)

# Assume ticker_history is a pandas DataFrame with the necessary columns
# Add the Future Close column
ticker_history.fillna(0, inplace=True)
ticker_history['Future Close'] = ticker_history['Close'].shift(-1)

# Create the datasets
scaler = StandardScaler()
scaled_features = scaler.fit_transform(ticker_history[['Open', 'High', 'Low', 'Close', 'Volume']])

# Create the datasets
appl_x_dataset = create_sequences(scaled_features, 50)

# The target should be the 'Future Close' for each sequence
appl_y_dataset = ticker_history['Future Close'].shift(-50).dropna()

# Only keep the samples where 'Future Close' is not NaN
appl_x_dataset = appl_x_dataset[:len(appl_y_dataset)]

# Define the model
model = keras.Sequential()
model.add(keras.layers.LSTM(50, activation='relu', input_shape=(50, 5), return_sequences=True))
model.add(keras.layers.LSTM(50, activation='relu'))
model.add(keras.layers.Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Summary of the model
model.summary()

# Fit the model
model.fit(appl_x_dataset, appl_y_dataset, epochs=1)

c:\Users\ppiot\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50, 50)         │        11,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,451 (122.86 KB)

 Trainable params: 31,451 (122.86 KB)

 Non-trainable params: 0 (0.00 B)

343/343 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 2687.4177


In [3]:
model.fit(appl_x_dataset, appl_y_dataset, epochs=1)

343/343 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 476.8351


In [4]:
model.fit(appl_x_dataset, appl_y_dataset, epochs=100)

Epoch 1/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 32.5495
Epoch 2/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 29.0019
Epoch 3/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 27.5008
Epoch 4/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 24.5498
Epoch 5/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 15107374080.0000
Epoch 6/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 162.1394
Epoch 7/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 2187.6370
Epoch 8/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 27.5869
Epoch 9/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 28.1710
Epoch 10/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 27.8726
Epoch 11/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 24.4836
Epoch 12/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 24.5409
Epoch 13/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 27.4689
Epoch 14/100
343/343 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 25.4982
Epo